# Day 3

In [3]:
import kotlin.io.path.Path
import kotlin.io.path.readLines

data class Position(
    val x: Int,
    val y: Int
)

data class NumberToken(
    val value: Int,
    val length: Int,
    val origin: Position,
)

data class SymbolToken(
    val symbol: String,
    val position: Position
)

val SymbolToken.neighbourPositions: Set<Position>
    get() =
        (-1..1).flatMap { x ->
            (-1..1).map { y ->
                Position(position.x + x, position.y + y)
            }
        }.toSet()

val NumberToken.neighbourPositions: Set<Position>
    get() =
        (-1..length).flatMap { x ->
            (-1..1).map { y ->
                Position(origin.x + x, origin.y + y)
            }
        }.toSet()

val NumberToken.occupiedPositions: Set<Position>
    get() =
        (0..<length).map {
            Position(origin.x + it, origin.y)
        }.toSet()

fun parseInput(lines: List<String>): Pair<Set<NumberToken>, Set<SymbolToken>> {
    data class ReductionState(
        val numbers: Set<NumberToken> = emptySet(),
        val symbols: Set<SymbolToken> = emptySet(),
        val currentNumberToken: NumberToken? = null,
    )

    return lines.mapIndexed { y, line ->
        val result = line.foldIndexed(ReductionState()) { x, state, character ->
            when {
                character.isDigit() -> state.copy(
                    currentNumberToken = state.currentNumberToken
                        ?.let {
                            it.copy(
                                value = it.value * 10 + character.digitToInt(),
                                length = it.length + 1
                            )
                        }
                        ?: NumberToken(
                            value = character.digitToInt(),
                            length = 1,
                            origin = Position(x, y)
                        )
                )

                character == '.' -> state.copy(
                    numbers = state.numbers + setOfNotNull(state.currentNumberToken),
                    currentNumberToken = null
                )


                else -> state.copy(
                    numbers = state.numbers + setOfNotNull(state.currentNumberToken),
                    symbols = state.symbols + SymbolToken(character.toString(), position = Position(x, y)),
                    currentNumberToken = null
                )
            }
        }

        val numbers = result.numbers + setOfNotNull(result.currentNumberToken)
        val symbols = result.symbols

        numbers to symbols
    }
        .reduce { (accumulatedNumbers, accumulatedSymbols), (numbers, symbols) ->
            (accumulatedNumbers + numbers) to (accumulatedSymbols + symbols)
        }
}

val lines = Path("./input").readLines().filter { it.isNotBlank() }
val (numbers, symbols) = parseInput(lines)


## Part 1

In [4]:
val symbolPositions = symbols.map { it.position }.toSet()

numbers
    .filter { number -> number.neighbourPositions.any { it in symbolPositions } }
    .sumOf { it.value }


543867

## Part 2

In [6]:
val numbersMap = numbers
    .flatMap { token -> token.occupiedPositions.map { it to token.value } }
    .fold(emptyMap<Position, Int>()) { map, positions -> map + positions }

symbols
    .filter { it.symbol == "*" }
    .sumOf { symbol ->
        val neighbourNumbers = symbol.neighbourPositions.mapNotNull { numbersMap[it] }.toSet()
        if (neighbourNumbers.size == 2) neighbourNumbers.reduce { a, b -> a * b } else 0
    }


79613331